In [ ]:
! pip install imdbPY

# LSTM

In [2]:
pip install tensorflow

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Note: you may need to restart the kernel to use updated packages.


# lstm 256 patch size adam w 

In [13]:
import pandas as pd
import re
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.metrics.pairwise import cosine_similarity

# Load the movie dataset
data = pd.read_csv(r'/kaggle/input/movies-updated/movies_with_tags_rating.csv')

# Preprocessing
df = pd.DataFrame(data)
df['overview'] = df['overview'].fillna('')
df['cast'] = df['cast'].fillna('')
df['director'] = df['director'].fillna('(no data about the director is found)')
df['genre'] = df['genre'].fillna('')
df['tag'] = df['tag'].fillna('')
df['reviews'] = df['reviews'].fillna('')

# Combine features into a single column for text processing
df['combined_features'] = df['overview'] + ' ' + df['cast'] + ' ' + df['director'] + ' ' + df['genre'] + ' ' + df['tag'] + ' ' + df['reviews']

# Tokenization and padding
max_words = 10000
max_sequence_len = 300

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(df['combined_features'])
sequences = tokenizer.texts_to_sequences(df['combined_features'])
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_len, padding='post', truncating='post')

# Define LSTM model
embedding_dim = 100

model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=max_sequence_len))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='AdamW', metrics=['accuracy'])
model.summary()

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, df['rating'], test_size=0.2, random_state=42)

# Train the LSTM model
model.fit(X_train, y_train, epochs=5, batch_size=256, validation_data=(X_test, y_test))

# Generate embeddings from the trained LSTM model
embedding_layer = Sequential()
embedding_layer.add(model.layers[0])
embedding_layer.add(model.layers[1])
embedding_layer.compile(loss='binary_crossentropy', optimizer='AdamW')

# Generate embeddings for all movies
movie_embeddings = embedding_layer.predict(padded_sequences)

# Flatten the LSTM output by averaging over timesteps
movie_embeddings_2d = np.mean(movie_embeddings, axis=1)

# Calculate cosine similarity based on 2D embeddings
cosine_sim_lstm = cosine_similarity(movie_embeddings_2d, movie_embeddings_2d)

# Function to clean movie titles (remove special characters and lowercase)
def extract_title(title):
    # Check if the title is NaN or not a string, return an empty string in that case
    if isinstance(title, str):
        title_cleaned = title.lower()
        title_cleaned = re.sub(r'\W+', ' ', title_cleaned)
        return title_cleaned.strip()
    else:
        return np.nan

# Function to recommend movies based on LSTM embeddings
def recommend_movies_lstm(movie_title, cosine_sim=cosine_sim_lstm, df=df):
    movie_title_clean = extract_title(movie_title)
    
    # Check if the movie title exists in the dataset
    if movie_title_clean in df['title'].apply(extract_title).values:
        # Get the index of the movie that matches the title
        idx = df.index[df['title'].apply(extract_title) == movie_title_clean][0]
        
        # Get the pairwise similarity scores of all movies with that movie
        sim_scores = list(enumerate(cosine_sim[idx]))
        
        # Sort the movies based on similarity scores
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        
        # Get the top 3 most similar movies (excluding the input movie itself)
        recommended_movies = []
        for score in sim_scores[1:]:
            movie_data = df.iloc[score[0]]
            recommended_movies.append({
                'title': movie_data['title'],
                'film_rate': movie_data['film_rate'],
                'imdbId': movie_data['imdbId'],
                'cover_url': movie_data['cover_url'],
                'director': movie_data['director'],
                'genre': movie_data['genre'],
                'cast': movie_data['cast'],
                'overview': movie_data['overview'],
                'reviews': movie_data['reviews'],
                'tag': movie_data['tag'],
                'rating': movie_data['rating']
            })
            if len(recommended_movies) == 3:  # Limit to top 3 movies
                break
        
        # Return the top 3 most similar movies with all requested details
        return recommended_movies
    else:
        return []  # Return an empty list if the movie is not found
# Example usage with LSTM model
test_movie = 'heat'
recommended_movies_lstm = recommend_movies_lstm(test_movie)

if not recommended_movies_lstm:
    print(f"'{test_movie}' is not listed in our database.")
else:
    print(f"Movies recommended based on '{test_movie}':")
    for movie in recommended_movies_lstm:
        print(f"Title: {movie['title']}, Rating: {movie['film_rate']}, User Rating: {movie['rating']}, Tags: {movie['tag']}")
        print(f"Director: {movie['director']}")
        print(f"Genre: {movie['genre']}")
        print("-" * 80)


/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_14"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_7 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_14 (LSTM)                  │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_15 (LSTM)                  │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
33/33 ━━━━━━━━━━━━━━━━━━━━ 5s 75ms/step - accuracy: 5.9395e-04 - loss: -2.3343 - val_accuracy: 0.0024 - val_loss: -14.6580
Epoch 2/5
33/33 ━━━━━━━━━━━━━━━━━━━━ 2s 65ms/step - accuracy: 5.0738e-04 - loss: -19.2410 - val_accuracy: 0.0024 - val_loss: -35.8940
Epoch 3/5
33/33 ━━━━━━━━━━━━━━━━━━━━ 2s 66ms/step - accuracy: 8.8278e-04 - loss: -44.2356 - val_accuracy: 0.0024 - val_loss: -74.9139
Epoch 4/5
33/33 ━━━━━━━━━━━━━━━━━━━━ 2s 65ms/step - accuracy: 4.3675e-04 - loss: -89.7688 - val_accuracy: 0.0024 - val_loss: -141.4596
Epoch 5/5
33/33 ━━━━━━━━━━━━━━━━━━━━ 2s 65ms/step - accuracy: 2.9908e-04 - loss: -165.9277 - val_accuracy: 0.0024 - val_loss: -245.6596
330/330 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step
Movies recommended based on 'heat':
Title: Hope Springs, Rating: 6.3, User Rating: 3.15, Tags: 
Director: David Frankel
Genre: Comedy, Drama, Romance
--------------------------------------------------------------------------------
Title: The Prophecy, Rating: 6.4, User Rating: 3.2, Tags

# lstm 256 batch adam

In [ ]:
import pandas as pd
import re
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.metrics.pairwise import cosine_similarity

# Load the movie dataset
data = pd.read_csv(r'/kaggle/input/movies-updated/movies_with_tags_rating.csv')

# Preprocessing
df = pd.DataFrame(data)
df['overview'] = df['overview'].fillna('')
df['cast'] = df['cast'].fillna('')
df['director'] = df['director'].fillna('(no data about the director is found)')
df['genre'] = df['genre'].fillna('')
df['tag'] = df['tag'].fillna('')
df['reviews'] = df['reviews'].fillna('')

# Combine features into a single column for text processing
df['combined_features'] = df['overview'] + ' ' + df['cast'] + ' ' + df['director'] + ' ' + df['genre'] + ' ' + df['tag'] + ' ' + df['reviews']

# Tokenization and padding
max_words = 10000
max_sequence_len = 300

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(df['combined_features'])
sequences = tokenizer.texts_to_sequences(df['combined_features'])
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_len, padding='post', truncating='post')

# Define LSTM model
embedding_dim = 100

model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=max_sequence_len))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, df['rating'], test_size=0.2, random_state=42)

# Train the LSTM model
model.fit(X_train, y_train, epochs=5, batch_size=256, validation_data=(X_test, y_test))

# Generate embeddings from the trained LSTM model
embedding_layer = Sequential()
embedding_layer.add(model.layers[0])
embedding_layer.add(model.layers[1])
embedding_layer.compile(loss='binary_crossentropy', optimizer='Adam')

# Generate embeddings for all movies
movie_embeddings = embedding_layer.predict(padded_sequences)

# Flatten the LSTM output by averaging over timesteps
movie_embeddings_2d = np.mean(movie_embeddings, axis=1)

# Calculate cosine similarity based on 2D embeddings
cosine_sim_lstm = cosine_similarity(movie_embeddings_2d, movie_embeddings_2d)

# Function to clean movie titles (remove special characters and lowercase)
def extract_title(title):
    # Check if the title is NaN or not a string, return an empty string in that case
    if isinstance(title, str):
        title_cleaned = title.lower()
        title_cleaned = re.sub(r'\W+', ' ', title_cleaned)
        return title_cleaned.strip()
    else:
        return np.nan

# Function to recommend movies based on LSTM embeddings
def recommend_movies_lstm(movie_title, cosine_sim=cosine_sim_lstm, df=df):
    movie_title_clean = extract_title(movie_title)
    
    # Check if the movie title exists in the dataset
    if movie_title_clean in df['title'].apply(extract_title).values:
        # Get the index of the movie that matches the title
        idx = df.index[df['title'].apply(extract_title) == movie_title_clean][0]
        
        # Get the pairwise similarity scores of all movies with that movie
        sim_scores = list(enumerate(cosine_sim[idx]))
        
        # Sort the movies based on similarity scores
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        
        # Get the top 3 most similar movies (excluding the input movie itself)
        recommended_movies = []
        for score in sim_scores[1:]:
            movie_data = df.iloc[score[0]]
            recommended_movies.append({
                'title': movie_data['title'],
                'film_rate': movie_data['film_rate'],
                'imdbId': movie_data['imdbId'],
                'cover_url': movie_data['cover_url'],
                'director': movie_data['director'],
                'genre': movie_data['genre'],
                'cast': movie_data['cast'],
                'overview': movie_data['overview'],
                'reviews': movie_data['reviews'],
                'tag': movie_data['tag'],
                'rating': movie_data['rating']
            })
            if len(recommended_movies) == 3:  # Limit to top 3 movies
                break
        
        # Return the top 3 most similar movies with all requested details
        return recommended_movies
    else:
        return []  # Return an empty list if the movie is not found
# Example usage with LSTM model
test_movie = 'heat'
recommended_movies_lstm = recommend_movies_lstm(test_movie)

if not recommended_movies_lstm:
    print(f"'{test_movie}' is not listed in our database.")
else:
    print(f"Movies recommended based on '{test_movie}':")
    for movie in recommended_movies_lstm:
        print(f"Title: {movie['title']}, Rating: {movie['film_rate']}, User Rating: {movie['rating']}, Tags: {movie['tag']}")
        print(f"Director: {movie['director']}")
        print(f"Genre: {movie['genre']}")
        print("-" * 80)


# another try 

In [17]:
import pandas as pd
import re
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.metrics.pairwise import cosine_similarity

# Load the movie dataset
data = pd.read_csv(r'/kaggle/input/movies-updated/movies_with_tags_rating.csv')

# Preprocessing
df = pd.DataFrame(data)
df['overview'] = df['overview'].fillna('')
df['cast'] = df['cast'].fillna('')
df['director'] = df['director'].fillna('(no data about the director is found)')
df['genre'] = df['genre'].fillna('')
df['tag'] = df['tag'].fillna('')
df['reviews'] = df['reviews'].fillna('')

# Feature Weighting
df['combined_features'] = (df['overview'] * 1 + ' ' +
                           df['cast'] * 3 + ' ' + 
                           df['director'] * 4 + ' ' + 
                           df['genre'] * 5 + ' ' + 
                           df['tag'] * 2 + ' ' + 
                           df['reviews'] * 1)
# Tokenization and padding
max_words = 10000
max_sequence_len = 300

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(df['combined_features'])
sequences = tokenizer.texts_to_sequences(df['combined_features'])
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_len, padding='post', truncating='post')

# Define LSTM model
embedding_dim = 100

model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=max_sequence_len))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, df['rating'], test_size=0.2, random_state=42)

# Train the LSTM model
model.fit(X_train, y_train, epochs=5, batch_size=256, validation_data=(X_test, y_test))

# Generate embeddings from the trained LSTM model
embedding_layer = Sequential()
embedding_layer.add(model.layers[0])
embedding_layer.add(model.layers[1])
embedding_layer.compile(loss='binary_crossentropy', optimizer='Adam')

# Generate embeddings for all movies
movie_embeddings = embedding_layer.predict(padded_sequences)

# Flatten the LSTM output by averaging over timesteps
movie_embeddings_2d = np.mean(movie_embeddings, axis=1)

# Calculate cosine similarity based on 2D embeddings
cosine_sim_lstm = cosine_similarity(movie_embeddings_2d, movie_embeddings_2d)

# Function to clean movie titles (remove special characters and lowercase)
def extract_title(title):
    if isinstance(title, str):
        title_cleaned = title.lower()
        title_cleaned = re.sub(r'\W+', ' ', title_cleaned)
        return title_cleaned.strip()
    else:
        return np.nan

# Function to recommend movies based on LSTM embeddings
def recommend_movies_lstm(movie_title, cosine_sim=cosine_sim_lstm, df=df):
    movie_title_clean = extract_title(movie_title)
    
    # Check if the movie title exists in the dataset
    if movie_title_clean in df['title'].apply(extract_title).values:
        # Get the index of the movie that matches the title
        idx = df.index[df['title'].apply(extract_title) == movie_title_clean][0]
        
        # Get the pairwise similarity scores of all movies with that movie
        sim_scores = list(enumerate(cosine_sim[idx]))
        
        # Sort the movies based on similarity scores
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        
        # Get the top 3 most similar movies (excluding the input movie itself)
        recommended_movies = []
        for score in sim_scores[1:]:
            movie_data = df.iloc[score[0]]
            recommended_movies.append({
                'title': movie_data['title'],
                'film_rate': movie_data['film_rate'],
                'imdbId': movie_data['imdbId'],
                'cover_url': movie_data['cover_url'],
                'director': movie_data['director'],
                'genre': movie_data['genre'],
                'cast': movie_data['cast'],
                'overview': movie_data['overview'],
                'reviews': movie_data['reviews'],
                'tag': movie_data['tag'],
                'rating': movie_data['rating']
            })
            if len(recommended_movies) == 3:  # Limit to top 3 movies
                break
        
        # Return the top 3 most similar movies with all requested details
        return recommended_movies
    else:
        return []  # Return an empty list if the movie is not found

# Example usage with LSTM model
test_movie = 'heat'
recommended_movies_lstm = recommend_movies_lstm(test_movie)

if not recommended_movies_lstm:
    print(f"'{test_movie}' is not listed in our database.")
else:
    print(f"Movies recommended based on '{test_movie}':")
    for movie in recommended_movies_lstm:
        print(f"Title: {movie['title']}, Rating: {movie['film_rate']}, User Rating: {movie['rating']}, Tags: {movie['tag']}")
        print(f"Director: {movie['director']}")
        print(f"Genre: {movie['genre']}")
        print("-" * 80)


/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_18"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_9 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_18 (LSTM)                  │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_19 (LSTM)                  │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_27 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_28 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_29 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
33/33 ━━━━━━━━━━━━━━━━━━━━ 5s 75ms/step - accuracy: 6.4984e-04 - loss: -2.4309 - val_accuracy: 0.0024 - val_loss: -16.3625
Epoch 2/5
33/33 ━━━━━━━━━━━━━━━━━━━━ 2s 65ms/step - accuracy: 0.0011 - loss: -22.1989 - val_accuracy: 0.0024 - val_loss: -43.5638
Epoch 3/5
33/33 ━━━━━━━━━━━━━━━━━━━━ 2s 65ms/step - accuracy: 3.4469e-04 - loss: -54.4752 - val_accuracy: 0.0024 - val_loss: -92.9786
Epoch 4/5
33/33 ━━━━━━━━━━━━━━━━━━━━ 2s 65ms/step - accuracy: 9.0351e-04 - loss: -112.7620 - val_accuracy: 0.0024 - val_loss: -181.9569
Epoch 5/5
33/33 ━━━━━━━━━━━━━━━━━━━━ 2s 65ms/step - accuracy: 4.4829e-04 - loss: -215.2816 - val_accuracy: 0.0024 - val_loss: -328.8805
330/330 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step
Movies recommended based on 'heat':
Title: Little Women, Rating: 7.3, User Rating: 3.65, Tags: 
Director: George Cukor
Genre: Drama, Family, Romance, War
--------------------------------------------------------------------------------
Title: Little Women, Rating: 7.3, User Rating: 3.65, Ta

In [18]:
test_movie = 'interstellar'
recommended_movies_lstm = recommend_movies_lstm(test_movie)

if not recommended_movies_lstm:
    print(f"'{test_movie}' is not listed in our database.")
else:
    print(f"Movies recommended based on '{test_movie}':")
    for movie in recommended_movies_lstm:
        print(f"Title: {movie['title']}, Rating: {movie['film_rate']}, User Rating: {movie['rating']}, Tags: {movie['tag']}")
        print(f"Director: {movie['director']}")
        print(f"Genre: {movie['genre']}")
        print("-" * 80)

Movies recommended based on 'interstellar':
Title: Material Girls, Rating: 3.9, User Rating: 1.95, Tags: 
Director: Martha Coolidge
Genre: Comedy, Family, Romance
--------------------------------------------------------------------------------
Title: Babe, Rating: 6.9, User Rating: 3.45, Tags: 
Director: Chris Noonan
Genre: Comedy, Drama, Family
--------------------------------------------------------------------------------
Title: How to Make an American Quilt, Rating: 6.3, User Rating: 3.15, Tags: 
Director: Jocelyn Moorhouse
Genre: Comedy, Drama, Romance
--------------------------------------------------------------------------------
